<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/multitaxa/multitaxa_inspect_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Compare model outputs with known crops
---   
*Last Updated 13 Feb 2025*  
-Now runs in Python 3 with Tensorflow 2.0-     

Compare trained model predicted and post-processed square crops with known square crops from EOL user generated test data.

Use trained object detection models to automatically crop images of snakes & lizards (Squamata), beetles (Coleoptera), frogs (Anura), and carnivores (Carnivora) to square dimensions centered around animal(s).

Models were trained and saved to Google Drive in [multitaxa_train_tf2_rcnns.ipynb](https://github.com/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_cropping/multitaxa/multitaxa_train_tf2_rcnns.ipynb).

***Models were trained in Python 2 and TF 1 in April 2020: Faster RCNN ResNet 50 trained for 12 hours to 200,000 steps and Faster RCNN Inception v2 for 18 hours to 200,000 steps.***

Notes:   
* Run code blocks by pressing play button in brackets on left
* Before you you start: change the runtime to "GPU" with "High RAM"
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

References:     
* [Official Tensorflow Object Detection API Instructions](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html)   
* [Medium Blog on training using Tensorflow Object Detection API in Colab](https://medium.com/analytics-vidhya/training-an-object-detection-model-with-tensorflow-api-using-google-colab-4f9a688d5e8b)

## Installs & Imports
---

In [ ]:
#@title Choose where to save results
# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# Mount google drive to export image cropping coordinate file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Note: You can modify "filter" to choose detection results for any class of interest the model is trained on
filter = "Multitaxa" # @param ["Multitaxa"] {"allow-input":true}

# Type in the path to your project wd in form field on right
basewd = "/content/drive/MyDrive/train" #@param ["/content/drive/MyDrive/train"] {allow-input: true}
# Type in the folder that you want to contain TF2 files
folder = "tf2" #@param ["tf2"] {allow-input: true}
# Define current working directory using form field inputs
cwd = basewd + '/' + folder + '/' + filter

# Install dependencies
!pip3 install --upgrade gdown
!gdown 1-0F-zKYOAV1qk2hu7kqKvlMo5Xsj6Uut # Download helper_funcs folder
!tar -xzvf multitaxa_helper_funcs.tar.gz -C .
#!pip install -r requirements.txt

In [ ]:
# TO DO figure out requirements.txt and fix warnings
!pip install numpy==1.24.3
!pip install protobuf==3.20.3

In [ ]:
#@title Choose model parameters, set up directory structure, and build Tensorflow Object Detection API

# Use EOL pre-trained model for object detection?
use_EOL_model = True #@param {type: "boolean"}

# If using your own trained model, change values to match your trained model
filters = ["Anura", "Carnivora", "Coleoptera", "Squamata"] #@param ["[\"Anura\", \"Carnivora\", \"Coleoptera\", \"Squamata\"]"] {type:"raw", allow-input: true}
PATH_TO_LABELS = "labelmap.pbtxt" #@param {type:"string"}
NUM_CLASSES = 4 #@param
saved_models_dir = "tf_models/train_demo/rcnn_i/finetuned_model/" #@param ["tf_models/train_demo/rcnn/finetuned_model/"] {allow-input: true}
mod_abbv = "rcnn_i"

# For working with directories
import os
import pathlib

# For downloading and displaying images
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import io
import scipy.misc
import numpy as np
import pandas as pd
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

# For object detection
import tensorflow as tf
import tensorflow_hub as hub
tf.get_logger().setLevel('ERROR')

# Import EOL custom helper_funcs
from setup import *
from wrangle_data import *

# Clone Tensorflow Object Detection Github Repo
setup_dirs(cwd)

# Download Multitaxa_crops_test.tsv
%cd $cwd
%cd results
!gdown 18SRUuOxKvLdJOt3fq9z_jmfBpVBbFLF2
%cd $cwd

# Build Tensorflow Object Detection API
!sudo apt install -y protobuf-compiler
!cd tf_models/models/research/ && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .
%cd $cwd

In [ ]:
# Build saved model

# For object detection
import sys
sys.path.append('/content')
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# For downloading and displaying images
import cv2
import tempfile
import urllib
from collections import defaultdict
from io import StringIO
from IPython.display import display

# For drawing onto images
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring inference time
import time

# For working with data
import subprocess
import csv
import tarfile
import zipfile

# Print Tensorflow version
print('\nTensorflow Version: %s' % tf.__version__)

# Check available GPU devices
print('The following GPU devices are available: %s' % tf.test.gpu_device_name())

# Unpack EOL saved model
PATH_TO_CKPT = saved_models_dir + 'frozen_inference_graph.pb'
detector = detection_graph = unpack_EOL_model(use_EOL_model, saved_models_dir, PATH_TO_CKPT, cwd)

# Load saved model and label map
print("\nLoading label map for {} class(es) from: \n{}".format(NUM_CLASSES, PATH_TO_LABELS))
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Generate crops: Run inference on EOL images & save resulting coordinates for cropping - 1K images
---
Use 20K EOL image bundle to generate bounding boxes around each object with pre-trained object detection models. Results are saved to [crops_file].tsv.

In [ ]:
#@title Define functions
%matplotlib inline

# Set the maximum number of detections to keep per image
max_boxes = 10 #@param {type:"slider", min:0, max:100, step:10}

# Set the minimum confidence score for detections to keep per image
min_score = 0.6 #@param {type:"slider", min:0, max:0.9, step:0.1}

# Set filename for saving classification results
def set_outpath(crops_file, cwd):
    outpath = cwd + '/' + 'results/' + os.path.splitext(crops_file)[0] + '_' + mod_abbv + '.tsv'
    print("\nSaving results to: \n", outpath)

    return outpath

# Export object detection results
def export_results(image_url, result, outfpath, im_h, im_w, filter=filters):
    with open(outfpath, 'a') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        img_id = os.path.splitext((os.path.basename(image_url)))
        # Write one row per detected object with bounding box coordinates
        num_detections = min(int(result["num_detections"][0]), max_boxes)
        for i in range(0, num_detections):
            class_name = category_index[result["detection_classes"][0][i]]['name']
            if any(fil in class_name for fil in filters): # Only writes rows for filtered class
                ymin = result["detection_boxes"][0][i][0]
                xmin = result["detection_boxes"][0][i][1]
                ymax = result["detection_boxes"][0][i][2]
                xmax = result["detection_boxes"][0][i][3]
                confidence = result["detection_scores"][0][i]
                tsv_writer.writerow([img_id, class_name, confidence,
                          xmin, ymin, xmax, ymax, im_h, im_w, image_url])
        print("\nObject detection results for Image {} saved to: {}".format(image_url, outfpath))

    return img_id

# Format cropping dimensions to EOL standards
def format_crops_for_eol(df):
# {"height":"423","width":"640","crop_x":123.712,"crop_y":53.4249,"crop_width":352,"crop_height":0}
    df['crop_dimensions'] = np.nan
    for i, row in df.iterrows():
        df.loc[i, 'crop_dimensions'] = ('{{"height":"{}","width":"{}","crop_x":{},"crop_y":{},"crop_width":{},"crop_height":{}}}'
        .format(df.im_height[i], df.im_width[i], df.xmin[i], df.ymin[i], df.crop_width[i], df.crop_height[i]))

    # Add other dataframe elements from cols: identifier, dataobjectversionid, eolmediaurl, im_class, crop_dimensions
    eol_crops = pd.DataFrame(df.iloc[:,np.r_[-5,-4,-6,0,-1]])
    print("\n EOL formatted cropping dimensions: \n", eol_crops.head())

    return eol_crops

print('Model loaded and functions defined! \nGo to next steps to run inference on images.')

### Generate predictions (crops): Run inference on EOL test images with known ground truths
Use EOL Multitaxa test images to get bounding boxes of detected animals. Results are saved to [crops_file].tsv.

In [ ]:
#@title Enter EOL cropping test image tsv and choose inference settings
%cd $cwd

# Load in EOL image bundle
bundle = "Multitaxa_crops_test.tsv" #@param {type:"string"}
bundle_path = 'results/' + bundle
df1 = read_datafile(bundle_path, sep='\t', header=0, disp_head=False)
ground_truth = df1.copy()
df = pd.DataFrame(df1['obj_url'])
df.columns = ['url']
print('\n EOL test images head:\n{}'.format(df.head()))

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for all images"]

# Display detection results on images?
if 'tiny subset' in run:
    display_results = True
else:
    display_results = False

# Run inference on EOL test images (with known ground truths)
crops_file = "multitaxa_cropcoords_tf2_d" #@param ["multitaxa_cropcoords_tf2_a", "multitaxa_cropcoords_tf2_b", "multitaxa_cropcoords_tf2_c", "multitaxa_cropcoords_tf2_d"] {allow-input: true}
outfpath = set_outpath(crops_file, cwd)

# Write header row of output tag file
if not os.path.isfile(outfpath):
    with open(outfpath, 'a') as out_file:
              tsv_writer = csv.writer(out_file, delimiter='\t')
              tsv_writer.writerow(["img_id", "class_name", "confidence", "xmin", \
                                   "ymin", "xmax", "ymax", "im_width", \
                                   "im_height", "url"])

In [ ]:
#@title Run EOL test images through trained model and save results
print("Running inference on images")
all_predictions = []
stop = len(df)
#stop = 5
start = 0
display_results = False
for i, row in enumerate(df.iloc[start:stop].iterrows()):
    try:
        # Run image through object detector and export result
        image_url = df['url'][row[0]]
        image_wboxes, result, im_h, im_w = run_detector_tf(detector, image_url, outfpath, filters, label_map, max_boxes, min_score, category_index)
        img_id = export_results(image_url, result, outfpath, im_h, im_w)

        # Optional: Display detections on images
        if (i+1<=50) and display_results:
            display_image(image_wboxes)

        # Display progress message after each image
        all_predictions.append(img_id)
        print('\033[92m {}) Inference complete for image {} of {} \033[0m \n'.format(i+1, i+1, stop))

    except:
        print('Check if URL from {} is valid\n'.format(df['url'][i]))

print("\n\n~~~\n\033[92m Inference complete!\033[0m \033[93m Run these steps for remaining batches A-D before proceeding.\033[0m\n~~~")

## Post-process detection results
---
Convert detection boxes into square, centered thumbnail cropping coordinates.

In [ ]:
#@title Enter path to inference result file

# If you just ran "Generate crops" above, you do not need to enter anything
# If you ran "Generate crops" during a previous session, enter the path for ONE output file
if 'outfpath' not in locals() or globals():
    crops_file = "multitaxa_cropcoords_tf2_d" #@param ["multitaxa_cropcoords_tf2_a", "multitaxa_cropcoords_tf2_b", "multitaxa_cropcoords_tf2_c", "multitaxa_cropcoords_tf2_d"] {allow-input: true}
    outfpath = set_outpath(crops_file, cwd)

df = pd.read_csv(outfpath, sep='\t', header=0, na_filter = False)
df.columns = ['img_id', 'class_name', 'confidence', 'xmin', 'ymin', 'xmax', 'ymax', 'im_width', 'im_height', 'url']
print("Bounding box dataframe, before post-processing: {} \n{}".format(outfpath, df.head()))

In [ ]:
# Add EOL img identifying info from breakdown file to cropping data
def add_identifiers(superboxes, bundle_info):
    # Get dataObjectVersionIDs, identifiers, and eolMediaURLS from indexed cols
    ids = pd.DataFrame(bundle_info[['data_object_id', 'obj_url']])
    ids.set_index('obj_url', inplace=True, drop=True)
    #print("Bundle identifying info head: \n", ids.head())

    # Set up superboxes df for mapping to bundle_info
    superboxes.reset_index(inplace=True)
    superboxes.rename(columns={'url': 'obj_url'}, inplace=True)
    superboxes.set_index('obj_url', inplace=True, drop=True)

    # Map dataObjectVersionIDs to crops_unq using eolMediaURL as the index
    crops_w_identifiers = pd.DataFrame(superboxes.merge(ids, left_index=True, right_index=True))
    crops_w_identifiers.reset_index(inplace=True)
    print("\n Crops with added EOL identifiers: \n", crops_w_identifiers.head())

    return crops_w_identifiers

In [ ]:
#@title Combine individual detection boxes into one "superbox" per image

# For images with >1 detection, make a 'super box' that containings all boxes

# Read in crop file exported from "Combine output files A-D" block above
crops = df.copy()
print(crops.head())

# De-normalize cropping coordinates to pixel values
crops = denormalize_coords(crops)
print(crops.head())

# Make 1 superbox per image [coordinates: bottom left (smallest xmin, ymin) and top right (largest xmax, ymax)]
superboxes = make_superboxes(crops)

# Read in EOL image "breakdown" bundle dataframe from "breakdown_download" bundle used for cropping
if 'bundle' not in locals() or globals():
    bundle = "Multitaxa_crops_test.tsv" #@param {type:"string"}
    bundle_path = 'results/' + bundle
bundle_info = read_datafile(bundle_path, sep='\t', header=0, disp_head=False)

# Add EOL img identifying info from breakdown file to cropping data
crops_w_identifiers = add_identifiers(superboxes, bundle_info)

In [ ]:
#@title Make superbox square and within image bounds (Optional: add padding)

# Pad by xx% larger crop dimension
pad = 0 #@param {type:"slider", min:0, max:10, step:2}
pad = pad/100 # Convert to percentage

# Make crops square and within bounds
df = make_square_crops(crops_w_identifiers, pad)

# Export crop coordinates to display_test.tsv to visualize results in next code block and confirm crop transformations
display_test_fpath = os.path.splitext(outfpath)[0] + '_displaytest' + '.tsv'
print("\n File for displaying square crops on images will be saved to: \n", display_test_fpath)
df.to_csv(display_test_fpath, sep='\t', index=False)

# Format image and cropping dimensions for EOL standards
eol_crops = format_crops_for_eol(df)

# Write results to tsv
eol_crops_fpath = os.path.splitext(display_test_fpath)[0].rsplit('_',2)[0] + '_20k_final' + '.tsv'
eol_crops.to_csv(eol_crops_fpath, columns = eol_crops.iloc[:,:-1], sep='\t', index=False)
print("EOL formatted crops dataset saved to: {} \n{}".format(eol_crops_fpath, eol_crops.head()))

In [ ]:
#@title Format EOL user crops to match display test crops

# Read in EOL image "breakdown" bundle dataframe from "breakdown_download" bundle used for cropping
if 'bundle' not in locals() or globals():
    bundle = "Multitaxa_crops_test.tsv" #@param {type:"string"}
    bundle_path = 'results/' + bundle
bundle_info = read_datafile(bundle_path, sep='\t', header=0, disp_head=False)
bundle_info.rename(columns={'name_x': 'class_name'}, inplace=True)

# Take and rename relevant columns
ground_truth = pd.DataFrame(bundle_info[['obj_url', 'im_height', 'im_width', 'xmin', 'ymin', 'xmax', 'ymax', 'class_name', 'data_object_id']])
ground_truth['crop_height'] = ground_truth['ymax'] - ground_truth['ymin']
ground_truth['crop_width'] = ground_truth['xmax'] - ground_truth['xmin']

# Export ground truth formatted crops
ground_truth_fpath = os.path.splitext(bundle_path)[0] + '_groundtruth' + '.tsv'
ground_truth.to_csv(ground_truth_fpath, sep='\t', index=False)
print("Ground truths saved to: ", ground_truth_fpath)
print(ground_truth.head())

## Display ground truths versus model predictions on images and save outputs
---
Images will have two boxes each - one ground truth and one model prediction

In [ ]:
#@title Read in cropping file and display results on images
from wrangle_data import *
import cv2

# Load in prediction df
pred_fname = "multitaxa_cropcoords_tf2_d_rcnn_i_displaytest.tsv" # @param ["Multitaxa_crops_test_groundtruth.tsv","multitaxa_cropcoords_tf2_d_rcnn_i_displaytest.tsv"] {"allow-input":true}
display_test_fpath = 'results/' + pred_fname
df = pd.read_csv(display_test_fpath, sep="\t", header=0)
df_pred = df.copy()

# Load in ground truth df
gt_fname = "Multitaxa_crops_test_groundtruth.tsv" # @param ["Multitaxa_crops_test_groundtruth.tsv","multitaxa_cropcoords_tf2_rcnn_displaytest.tsv"] {"allow-input":true}
display_test_fpath = 'results/' + gt_fname
df_gt = pd.read_csv(display_test_fpath, sep="\t", header=0)
df_gt = df_gt.set_index('obj_url')

print(df.head())
print(df_gt.head())
print(df_pred.head())

In [ ]:
# Draw cropping box on image
def draw_box_on_image(df, i, img, tags, tag_gt):
    # Get box coordinates
    xmin = df['xmin'][i].astype(int)
    ymin = df['ymin'][i].astype(int)
    xmax = df['xmax'][i].astype(int)
    ymax = df['ymax'][i].astype(int)
    boxcoords = [xmin, ymin, xmax, ymax]
    class_name = str(df['class_name'][i])
    if any(tag in class_name for tag in tags):
        tag = class_name + " " + str(tag_gt)
    else:
        tag = "None" + " " + str(tag_gt)

    # Set box/font color and size
    mindim = min(df['im_height'][i], df['im_width'][i])
    thicknessScale = 1e-3  # Adjust for larger thickness in all images
    fontScale = 2e-3
    fontsize = mindim * fontScale
    thickness = math.ceil(mindim * thicknessScale)
    box_col = (255, 0, 157)

    # Add label to image
    img = np.squeeze(img)
    buffer = int(mindim * 0.01) # some of the boxes are not drawing on images by edges,makes easier to see
    image_wbox = cv2.putText(img, tag, ((xmin+buffer), (ymax-buffer)), cv2.FONT_HERSHEY_SIMPLEX, fontsize, box_col, 2, cv2.LINE_AA)

    # Draw box label on image
    image_wbox = cv2.rectangle(img, ((xmin+buffer), (ymax-buffer)), ((xmax-buffer), (ymin+buffer)), box_col, thickness)

    return image_wbox, boxcoords

In [ ]:
#@title Draw bounding boxes on image & save resulting images
%matplotlib inline
import imageio

# Display results on images (only use for 5 images as a test)
display_results = False #@param {type:"boolean"}
print("Displaying results on images for small subset of 5 images. Uncheck display_results to run and save for all images.")

# Adjust line to right to see up to 50 images displayed at a time
start = 0 # @param {"type":"number","placeholder":"0"}
if display_results:
    stop = start + 5
else:
    stop = start + len(df)

# Loop through images
for i, row in df.iloc[start:stop].iterrows():
    try:
        # Read in image
        url = df['obj_url'][i]
        image, im_h, im_w = url_to_image(url)

        # Draw bounding box on image - Ground Truth
        image_wbox, boxcoords_gt = draw_box_on_image(df_gt, i, image, filters, "ground truth")

        # Draw bounding box on image - Prediction
        image_wboxes, boxcoords_pred = draw_box_on_image(df_pred, i, image_wbox, filters, "prediction")

        if display_results:
            # Plot cropping box on image
            _, ax = plt.subplots(figsize=(10, 10))
            ax.imshow(image_wboxes)

            # Display image URL and coordinates above image
            plt.title('{} \n im_h: {} im_w: {} \n Ground Truth - xmin: {}, ymin: {}, xmax: {}, ymax: {} \n Prediction - xmin: {}, ymin: {}, xmax: {}, ymax: {}'.format(url, im_h, im_w, boxcoords_gt[0], boxcoords_gt[1], boxcoords_gt[2], boxcoords_gt[3], boxcoords_pred[0], boxcoords_pred[1], boxcoords_pred[2], boxcoords_pred[3]))

        # Save Results
        path = "results/inspect_results/" + str(df['data_object_id'][i]) + "_wbox.png"
        imageio.imwrite(path, image_wboxes)
        print("Image with boxes saved to: ", path)

    except:
        pass

In [ ]:
unzipped_fpath = cwd + '/results/' + "inspect_results" #@param ["images.zip"] {allow-input: true}
zip_fpath = '/content/' + "inspect_results.zip" #@param ["images"] {allow-input: true}

# Zip the file/folder
!zip -r -j $zip_fpath $unzipped_fpath

## Caculate mAP for test images
---
Convert detection boxes into square, centered thumbnail cropping coordinates.

In [ ]:
#@title Convert cropping coordinates for EOL user generated ground truths (square crops) to mAP format
%cd $cwd

# Load in prediction df
pred_fname = "multitaxa_cropcoords_tf2_d_rcnn_i_displaytest.tsv" # @param ["Multitaxa_crops_test_groundtruth.tsv","multitaxa_cropcoords_tf2_d_rcnn_i_displaytest.tsv"] {"allow-input":true}
display_test_fpath = 'results/' + pred_fname
df = pd.read_csv(display_test_fpath, sep="\t", header=0)
df_pred = df.copy()

# Load in ground truth df
gt_fname = "Multitaxa_crops_test_groundtruth.tsv" # @param ["Multitaxa_crops_test_groundtruth.tsv","multitaxa_cropcoords_tf2_rcnn_displaytest.tsv"] {"allow-input":true}
display_test_fpath = 'results/' + gt_fname
df_gt = pd.read_csv(display_test_fpath, sep="\t", header=0)
df_gt = df_gt.set_index('obj_url')
df_gt.reset_index(inplace=True)

print("df", df.head())
print("gt", df_gt.head())
print("pred", df_pred.head())

df['xmin_gt'] = df_gt['xmin'].copy()
df['ymax_gt'] = df_gt['ymax'].copy()
df['xmax_gt'] = df_gt['xmax'].copy()
df['ymin_gt'] = df_gt['ymin'].copy()
df['class_name_gt'] = df_gt['class_name'].copy()
print("Df with new cols: ", df.head())

# Format bounding boxes for calculating mAP and IoU
pred_bboxes = []
pred_classes = []
pred_confs = []
gt_bboxes = []
gt_classes = []
# Loop through df and extract relevant info as lists
for i, row in df.iterrows():
    pred_bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax'], row['confidence']]
    pred_class = row['class_name']
    pred_conf = row['confidence']
    gt_bbox = [row['xmin_gt'], row['ymin_gt'], row['xmax_gt'], row['ymax_gt']]
    gt_class = row['class_name_gt']

    pred_bboxes.append(pred_bbox)
    pred_classes.append(pred_class)
    pred_confs.append(pred_conf)
    gt_bboxes.append(gt_bbox)
    gt_classes.append(gt_class)

# Reformatted bounding box data
print("\n\n~~~Reformatted bounding box data for mAP analysis~~~\n")
print("pred_bboxes", pred_bboxes)
print("pred_classes", pred_classes)
print("pred_confs", pred_confs)
print("gt_bboxes", gt_bboxes)
print("gt_classes", gt_classes)

In [ ]:
# Define functions
from sklearn.metrics import average_precision_score

# Calculate mean averag precision
def calculate_map(gt_boxes, pred_boxes, iou_threshold=0.5):
    aps = []
    y_true = []
    y_scores = []
    ious = []
    for i, gt_box in enumerate(gt_boxes):
          iou = calculate_iou(gt_box, pred_boxes[i])
          if iou > iou_threshold:
                y_true.append(1)
                y_scores.append(pred_boxes[i][4])  # Confidence score is at index 4
                ious.append(iou)

    if len(y_true) > 0:
            print("\n Number of true detections with iou > {} : {}: ".format(iou_threshold, len(y_true)))
            print("Number of total detections: ", len(gt_boxes))
            ap = average_precision_score(y_true, y_scores)
            aps.append(ap)

    return np.mean(aps), np.mean(ious)

# Calculate intersection over union
def calculate_iou(box1, box2):
    # Calculate intersection area
    print("box1: ", box1)
    print("box2: ", box2)
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    print("x1: ", x1)
    print("y1: ", y1)
    print("x2: ", x2)
    print("y2: ", y2)
    intersection_area = (x2 - x1) * (y2 - y1)
    if intersection_area < 0:
        intersection_area = 0
    print("intersection: ", intersection_area)

    # Calculate union area
    box1_area = abs((box1[2] - box1[0]) * (box1[3] - box1[1]))
    box2_area = abs((box2[2] - box2[0]) * (box2[3] - box2[1]))
    union_area = box1_area + box2_area - intersection_area
    print("box1_area: ", box1_area)
    print("box2_area: ", box2_area)
    print("union: ", union_area)

    # Calculate IoU
    iou = intersection_area / union_area
    print("iou: ", iou)
    print("\n")
    return iou

In [ ]:
#@title Calculate mAP
map, mIoU = calculate_map(gt_bboxes, pred_bboxes, iou_threshold=0.5)
print("\nmAP", map)
print("\nmIoU", mIoU)